In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/My Drive/College Work/Semester 7/EE698V/Project"

/content/drive/My Drive/College Work/Semester 7/EE698V/Project


In [ ]:
!pip install python-Levenshtein
!pip install librosa==0.8.0
# !pip install numpy==1.19
!pip install tensorflow

In [ ]:
import librosa
import pickle
import librosa.display
import IPython.display as ipd
import utils
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers, initializers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tqdm.notebook import tqdm
import numpy as np
from scipy.io.wavfile import write

In [ ]:
# Importing filenames and labels
labels_DF = pd.read_csv("labels_train.csv")
labels_DF.head()

,slice_file_name,class
0,100652-3-0-0.wav,dog_bark
1,100652-3-0-1.wav,dog_bark
2,100652-3-0-2.wav,dog_bark
3,100652-3-0-3.wav,dog_bark
4,101415-3-0-2.wav,dog_bark


In [ ]:
# audio_files = []
# labels = []
# spectrums = []
# cepstrums = []
# for i in tqdm(range(0,1761)):
#     name, label = labels_DF.iloc[i]
#     print(i, name, label)
#     audio, SR = librosa.load('audio_train/' + str(name))
#     audio_files = audio_files + [np.array(audio)]
#     labels = labels + [label]
#     spectrum = utils.wav2feat('audio_train/'+name)
#     spectrums.append(spectrum)
#     cepstrum = librosa.feature.melspectrogram(S = spectrum, sr = 44100, n_fft=1024, hop_length=441, win_length=882, window='hann', center=True, pad_mode='reflect')
#     cepstrums.append(cepstrum)

0 100652-3-0-0.wav dog_bark
1 100652-3-0-1.wav dog_bark
2 100652-3-0-2.wav dog_bark
3 100652-3-0-3.wav dog_bark
4 101415-3-0-2.wav dog_bark
5 101415-3-0-3.wav dog_bark
6 101415-3-0-8.wav dog_bark
7 102104-3-0-0.wav dog_bark
8 102106-3-0-0.wav dog_bark
9 102305-6-0-0.wav gun_shot
10 102842-3-0-1.wav dog_bark
11 102842-3-1-0.wav dog_bark
12 102842-3-1-5.wav dog_bark
13 102842-3-1-6.wav dog_bark
14 102858-5-0-0.wav engine_idling
15 102858-5-0-1.wav engine_idling
16 102871-8-0-0.wav siren
17 102871-8-0-1.wav siren
18 102871-8-0-10.wav siren
19 102871-8-0-11.wav siren
20 102871-8-0-12.wav siren
21 102871-8-0-13.wav siren
22 102871-8-0-14.wav siren
23 102871-8-0-15.wav siren
24 102871-8-0-2.wav siren
25 102871-8-0-3.wav siren
26 102871-8-0-4.wav siren
27 102871-8-0-5.wav siren
28 102871-8-0-6.wav siren
29 102871-8-0-7.wav siren
30 102871-8-0-8.wav siren
31 102871-8-0-9.wav siren
32 103074-7-0-0.wav jackhammer
33 103074-7-0-1.wav jackhammer
34 103074-7-0-2.wav jackhammer
35 103074-7-1-0.wav j

In [ ]:
with open('audio_files-labels-spectrums-cepstrums.pkl', 'rb') as f:
  audio_files = pickle.load(f)
  labels = pickle.load(f)
  spectrums = pickle.load(f)
cepstrums = []
for spectrum in spectrums:
  cepstrum = librosa.feature.melspectrogram(S = spectrum**2, sr = 44100, n_fft=1024, hop_length=441, win_length=882, window='hann', center=True, pad_mode='reflect')
  cepstrums.append(cepstrum)

In [ ]:
MFCCs = [librosa.feature.mfcc(S = librosa.power_to_db(spectrum**2), sr = 44100, n_mfcc = 100) for spectrum in tqdm(spectrums)]
features = [np.sum(MFCC, axis = 1)/MFCC.shape[1] for MFCC in MFCCs]
# features = [np.sum(spectrum, axis = 1)/spectrum.shape[1] for spectrum in spectrums]
features = np.array(features)
features = (features - np.mean(features, axis = 1).reshape([-1,1]))/np.std(features, axis = 1).reshape([-1,1])

In [ ]:
print(features.shape)

(1761, 100)


In [ ]:
lb = LabelEncoder()
lb.fit_transform(["dog_bark", "gun_shot", "engine_idling", "siren", "jackhammer", "drilling", "children_playing", "street_music", "air_conditioner", "car_horn"])

array([3, 6, 5, 8, 7, 4, 2, 9, 0, 1])

In [ ]:
one_hot_labels = pd.get_dummies(labels).to_numpy()
print(one_hot_labels.shape, one_hot_labels[0])

(1761, 10) [0 0 0 1 0 0 0 0 0 0]


In [ ]:
randomize = np.arange(len(features))
np.random.shuffle(randomize)
features = features[randomize]
one_hot_labels = one_hot_labels[randomize]

In [ ]:
model = Sequential()

model.add(Dense(256, input_shape=(100,)))
model.add(Activation('relu'))

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
model.fit(features, one_hot_labels, batch_size=512, epochs=300, validation_split = 0.2)

Epoch 1/300
3/3 [==============================] - 0s 66ms/step - loss: 2.3100 - accuracy: 0.1307 - val_loss: 2.2395 - val_accuracy: 0.1558
Epoch 2/300
3/3 [==============================] - 0s 18ms/step - loss: 2.2122 - accuracy: 0.1989 - val_loss: 2.1566 - val_accuracy: 0.2975
Epoch 3/300
3/3 [==============================] - 0s 16ms/step - loss: 2.1376 - accuracy: 0.2315 - val_loss: 2.0544 - val_accuracy: 0.3144
Epoch 4/300
3/3 [==============================] - 0s 15ms/step - loss: 2.0919 - accuracy: 0.2386 - val_loss: 1.9773 - val_accuracy: 0.3768
Epoch 5/300
3/3 [==============================] - 0s 15ms/step - loss: 2.0144 - accuracy: 0.2493 - val_loss: 1.9077 - val_accuracy: 0.3853
Epoch 6/300
3/3 [==============================] - 0s 16ms/step - loss: 1.9702 - accuracy: 0.2820 - val_loss: 1.8539 - val_accuracy: 0.3654
Epoch 7/300
3/3 [==============================] - 0s 16ms/step - loss: 1.9148 - accuracy: 0.3139 - val_loss: 1.7968 - val_accuracy: 0.3938
Epoch 8/300
3/3 [===

In [ ]:
model.save("Save/Task_1/v3.model")

INFO:tensorflow:Assets written to: Save/Task_1/v3.model/assets


In [ ]:
model = keras.models.load_model('Save/Task_1/v3.model')
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_96 (Dense)             (None, 256)               25856     
_________________________________________________________________
activation_91 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_97 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_92 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 128)               32896     
_________________________________________________________________
activation_93 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_99 (Dense)             (None, 128)             

# **Task 2**

In [ ]:
def predictSound(y, lb):
    MFCC = librosa.feature.mfcc(S = librosa.power_to_db(y), sr = 44100, n_mfcc = 100)
    feature = np.sum(MFCC, axis = 1)/MFCC.shape[1]
    feature = feature.T
    feature = (feature - np.mean(feature))/np.std(feature)
    feature = feature.T
    result = model.predict(feature.reshape([1, 100]))
    predictions = [np.argmax(y) for y in result]
    return lb.inverse_transform([predictions[0]])[0]

In [458]:
# dog_bark_DF = labels_DF[(labels_DF["class"] == "dog_bark")]
# gun_shot_DF = labels_DF[(labels_DF["class"] == "gun_shot")]
# engine_idling_DF = labels_DF[(labels_DF["class"] == "engine_idling")]
# siren_DF = labels_DF[(labels_DF["class"] == "siren")]
# jackhammer_DF = labels_DF[(labels_DF["class"] == "jackhammer")]
# drilling_DF = labels_DF[(labels_DF["class"] == "drilling")]
# children_playing_DF = labels_DF[(labels_DF["class"] == "children_playing")]
# street_music_DF = labels_DF[(labels_DF["class"] == "street_music")]
# air_conditioner_DF = labels_DF[(labels_DF["class"] == "air_conditioner")]
# car_horn_DF = labels_DF[(labels_DF["class"] == "car_horn")]

In [459]:
# %cd audio_train/
# dog_bark_sample = dog_bark_DF.sample().reset_index()["slice_file_name"][0]
# gun_shot_sample = gun_shot_DF.sample().reset_index()["slice_file_name"][0]
# engine_idling_sample = engine_idling_DF.sample().reset_index()["slice_file_name"][0]
# siren_sample = siren_DF.sample().reset_index()["slice_file_name"][0]
# jackhammer_sample = jackhammer_DF.sample().reset_index()["slice_file_name"][0]
# drilling_sample = drilling_DF.sample().reset_index()["slice_file_name"][0]
# children_playing_sample = children_playing_DF.sample().reset_index()["slice_file_name"][0]
# street_music_sample = street_music_DF.sample().reset_index()["slice_file_name"][0]
# air_conditioner_sample = air_conditioner_DF.sample().reset_index()["slice_file_name"][0]
# car_horn_sample = car_horn_DF.sample().reset_index()["slice_file_name"][0]

# lst = [dog_bark_sample,gun_shot_sample,engine_idling_sample,siren_sample,jackhammer_sample,drilling_sample,children_playing_sample,street_music_sample,air_conditioner_sample,car_horn_sample]

# lst = random.sample(lst, random.randrange(10))
# y_stft = utils.wavs2feat(lst)
# print(lst)

In [460]:
# # librosa.display.specshow(librosa.spectrum.amplitude_to_db(y_stft))
# gt = []
# for f in lst:
#   gt.append(labels_DF["class"][labels_DF.index[(labels_DF["slice_file_name"] == f)].to_list()[0]])
# print(gt)
# y = librosa.istft(y_stft)
# # ipd.Audio(y, rate=22050)

In [461]:
# # sound_clips = split_audio(y, w = 4000, h = 512, threshold_level = 0.1, tolerence=10)
# sound_clips = list(range(0, y_stft.shape[1], 86))
# if len(sound_clips)%2 != 0:
#     sound_clips.append(y_stft.shape[1]-1)
# a = []
# for i in range(0, len(sound_clips)-1):
#     a.append([sound_clips[i], sound_clips[i+1]])
# print(a)

In [462]:
# ans = []
# for intvl in a:
#     clip = y_stft[:, intvl[0]:intvl[1]]
#     check = predictSound(clip, lb)
#     if len(ans)==0 or ans[-1] != check:
#         ans.append(check)
# ans

In [463]:
# gt

In [464]:
# labels = [lb.inverse_transform([np.argmax(one_hot_label)])[0] for one_hot_label in one_hot_labels]

In [465]:
# features[1].shape

In [466]:
# predictions = [model.predict(feature.reshape([1, 100])) for feature in tqdm(features)]

In [467]:
# predictions_test = np.array([np.argmax(perdiction, axis=1) for prediction in predictions])

In [468]:
# predictions = [lb.inverse_transform([np.argmax(prediction)])[0] for prediction in predictions]

# **For** **Submission**

In [ ]:
%cd "/content/drive/My Drive/College Work/Semester 7/EE698V/Project"
model = keras.models.load_model('Save/Task_1/v3.model')

# Extracting Features
test_spectrums = []
test_mfcc = []

dict_labels = {'air_conditioner' : 0, 'car_horn' : 1, 'children_playing' : 2,\
               'dog_bark' : 3, 'drilling' : 4,'engine_idling' : 5, \
               'gun_shot' : 6, 'jackhammer' : 7, \
               'siren' : 8,'street_music' : 9}
list_labels = ['air_conditioner', 'car_horn', 'children_playing','dog_bark', 'drilling','engine_idling','gun_shot', 'jackhammer','siren','street_music']

file_names = []
test_spectrogram = []
for i in range(0, 50):
    path = "test_task1/test_task1/feats/"
    if(i<9):
      filename = "a00" + str(i+1) + ".npy"
    else:
      filename = "a0" + str(i+1) + ".npy"
    path = path + filename
    file_names.append(filename)
    with open(path, 'rb') as f:
      spectrum = np.load(f)
      test_spectrogram.append(spectrum)

MFCCs = [librosa.feature.mfcc(S = librosa.power_to_db(spectrum**2), sr = 44100, n_mfcc = 100) for spectrum in tqdm(test_spectrogram)]
test_mfcc = [np.sum(MFCC, axis = 1)/MFCC.shape[1] for MFCC in MFCCs]
# features = [np.sum(spectrum, axis = 1)/spectrum.shape[1] for spectrum in spectrums]
test_mfcc = np.array(test_mfcc)
test_mfcc = (test_mfcc - np.mean(test_mfcc, axis = 1).reshape([-1,1]))/np.std(test_mfcc, axis = 1).reshape([-1,1])

test_predictions_classes = model.predict_classes(np.array(test_mfcc))

est_labels = []
for i in range(0,50):
    est_labels.append(list_labels[test_predictions_classes[i]])

final_array = np.vstack(([np.array(file_names), np.array(est_labels)]))
print(final_array)
df = pd.DataFrame(final_array,).T
df.to_csv("est_task1.csv")

/content/drive/My Drive/College Work/Semester 7/EE698V/Project



[['a001.npy' 'a002.npy' 'a003.npy' 'a004.npy' 'a005.npy' 'a006.npy'
  'a007.npy' 'a008.npy' 'a009.npy' 'a010.npy' 'a011.npy' 'a012.npy'
  'a013.npy' 'a014.npy' 'a015.npy' 'a016.npy' 'a017.npy' 'a018.npy'
  'a019.npy' 'a020.npy' 'a021.npy' 'a022.npy' 'a023.npy' 'a024.npy'
  'a025.npy' 'a026.npy' 'a027.npy' 'a028.npy' 'a029.npy' 'a030.npy'
  'a031.npy' 'a032.npy' 'a033.npy' 'a034.npy' 'a035.npy' 'a036.npy'
  'a037.npy' 'a038.npy' 'a039.npy' 'a040.npy' 'a041.npy' 'a042.npy'
  'a043.npy' 'a044.npy' 'a045.npy' 'a046.npy' 'a047.npy' 'a048.npy'
  'a049.npy' 'a050.npy']
 ['children_playing' 'dog_bark' 'engine_idling' 'jackhammer' 'dog_bark'
  'children_playing' 'street_music' 'engine_idling' 'children_playing'
  'gun_shot' 'children_playing' 'engine_idling' 'children_playing'
  'street_music' 'air_conditioner' 'drilling' 'engine_idling' 'dog_bark'
  'drilling' 'children_playing' 'drilling' 'dog_bark' 'dog_bark'
  'street_music' 'air_conditioner' 'street_music' 'street_music'
  'dog_bark' 'str

/content/drive/My Drive/College Work/Semester 7/EE698V/Project


,0,1
0,a001.npy,siren-gun_shot-siren-drilling-jackhammer-gun_s...
1,a002.npy,street_music-drilling-children_playing-dog_bark
2,a003.npy,siren-drilling-street_music-children_playing-d...
3,a004.npy,siren-dog_bark-siren-children_playing-dog_bark
4,a005.npy,engine_idling-air_conditioner-car_horn


# Checking Part Two

In [ ]:
%cd "/content/drive/My Drive/College Work/Semester 7/EE698V/Project"
def task_predictions(y_stft):
  # sound_clips = split_audio(y, w = 4000, h = 512, threshold_level = 0.1, tolerence=10)
  sound_clips = list(range(0, y_stft.shape[1], 86))
  if len(sound_clips)%2 != 0:
      sound_clips.append(y_stft.shape[1]-1)
  a = []
  for i in range(0, len(sound_clips)-1):
      a.append([sound_clips[i], sound_clips[i+1]])
  #print(a)
  ans = []
  final = ""
  for intvl in a:
      clip = y_stft[:, intvl[0]:intvl[1]]
      check = predictSound(clip, lb)
      if len(ans)==0 or ans[-1] != check:
          if(len(ans) == 0):
              final = check
          else:
            final = final + '-' + check
          ans.append(check)
  return final

file_names = []
test_spectrogram = []
for i in range(0, 10):
    path = "sample_test_task2/feats/"
    if(i<9):
      filename = "a0000" + str(i+1) + ".npy"
    else:
      filename = "a000" + str(i+1) + ".npy"
    path = path + filename
    file_names.append(filename)
    with open(path, 'rb') as f:
      spectrum = np.load(f)
      test_spectrogram.append(spectrum)

answers_sample_task_2 = []
for i in range(0, 10):
    answers_sample_task_2.append(task_predictions(test_spectrogram[i]))

final_array1 = np.vstack(([np.array(file_names), np.array(answers_sample_task_2)]))
#print(final_array)
df1 = pd.DataFrame(final_array1).T
df1.head(), df.tail()

/content/drive/My Drive/College Work/Semester 7/EE698V/Project


(            0                                                  1
 0  a00001.npy    jackhammer-car_horn-children_playing-jackhammer
 1  a00002.npy                         car_horn-drilling-gun_shot
 2  a00003.npy                            drilling-car_horn-siren
 3  a00004.npy  jackhammer-street_music-jackhammer-dog_bark-gu...
 4  a00005.npy  jackhammer-children_playing-drilling-children_...,
            0                                                  1
 25  a026.npy                       drilling-jackhammer-drilling
 26  a027.npy  drilling-jackhammer-street_music-jackhammer-st...
 27  a028.npy    street_music-jackhammer-street_music-jackhammer
 28  a029.npy             dog_bark-street_music-jackhammer-siren
 29  a030.npy  children_playing-siren-car_horn-siren-children...)